In [1]:
import requests
from bs4 import BeautifulSoup
from numpy import nan
import pandas as pd

In [2]:
def getsoup(url):
    response = requests.get(url)
    print('response',response)
    print('url', url)
    html_doc = response.text
    soup = BeautifulSoup(html_doc,'html.parser')
    return soup


In [3]:
def elementdata(soup):
    data = {}

    try: 
        description = soup.find('h1',{'class':'product__header'}).string
        data['Description'] = description
    except AttributeError:
        data['Description'] = nan

    try:
        discount_price = soup.find('span',{'class':'product__price'}).string.strip()
        discount_price = discount_price.replace(',','')
        data['Discount_Price'] = float(discount_price.strip('$'))
    except AttributeError:
        data['Discount_Price'] = nan

    try:
        price = soup.find('span',{'class':'product__compare-at'}).string.strip()
        price = price.replace(',','')
        data['Price'] = float(price.strip('$'))
    except:
        data['Price'] = nan


    try:
        specs = soup.find('div',{'id':'toggle-product__description'})
        details = [detail.text for detail in specs.find_all('p')]
        details = [detail.text for detail in specs]
        print("details",details)
        stone_flag = False 
        stone_string = ""
    
        for item in range(len(details)):
  #          print("details",details[item])
            if 'Stones' in details[item]:
                stone_flag = True
                if ':' in details[item]:
  #                  print("Inside stone first flag true and : ")
                    try:
                        metal_col, metal_type = details[item].split(':')
                        data[metal_col] = metal_type.strip()
                    except ValueError:
                        print("ValueError: too many values to unpack (expected 2)", details[item])
                        pass
                continue
            
   #         print("No continue ")
            if stone_flag == True:
                if ':' in details[item]:
    #                print("Inside stone flag true and : ")
                    stone_flag = False
                else:
     #               print(print("Inside stone flag true and else "))
                    stone_string = details[item]
                    data[metal_col] = data[metal_col] + ',' + stone_string
                    
            if stone_flag != True:
                if ':' in details[item]:
      #              print("Inside stone flag false and : ")
                    try:
                        metal_col, metal_type = details[item].split(':')
                        data[metal_col] = metal_type.strip()
                    except ValueError:
                        total_semi = details[item].count(':')
                        
                        all_col = details[item].split('.')
                        
                        print("all_Col",all_col)
                        
                        for i in range(0, total_semi):
                            if ':' in all_col[i]:
                                try:
                                    metal_col, metal_type = all_col[i].split(':')
                                    data[metal_col] = metal_type.strip()
                                except (ValueError, IndexError):
                                    print("ValueError1: too many values to unpack (expected 2)",all_col[i])
                                    break
                            else:
                                data['Information'] = all_col[i].strip()
                else:
       #             print("Inside stone flag false and information ")
                    data['Information'] = details[item].strip()

    except AttributeError:
        pass

    return data

In [4]:
def getdata(filename):
    element_data = []
    with open(filename, "r") as file:
        for url in file:
            soup=getsoup(url.strip())
            element_data.append(elementdata(soup))
    return element_data

## Get Data for Earrings

In [ ]:
#df_earrings = pd.DataFrame(getdata('effy_earrings.txt'))

#df_earrings['Jewelry_Type'] = 'Earrings'
#df_earrings['Sex'] = 'Women' 
        
#df_earrings.count()

In [ ]:
#Copy the data to csv file 
#df_earrings.to_csv("effy_earrings.csv", index=False, header=True)

## Get data for Rings

In [ ]:
#get Rings data
#df_rings = pd.DataFrame(getdata('effy_rings.txt'))


In [ ]:

#df_rings['Jewelry_Type'] = 'Rings'
#df_rings['Sex'] = 'Women' 
           
#df_rings.head(1)

In [ ]:
#Copy data to csv file 
df_rings.to_csv("effy_rings.csv", index=False, header=True)


## Get data for Bracelets 

In [ ]:
#df_bracelets = pd.DataFrame(getdata('effy_bracelets.txt'))

#df_bracelets['Jewelry_Type'] = 'Bracelets'
#df_bracelets['Sex'] = 'Women' 



In [ ]:
df_bracelets.head(1)

In [ ]:

df_bracelets.to_csv("effy_bracelets.csv", index=False, header=True)


## Get Necklace and Pendant Data

In [ ]:

df_neck_pend = pd.DataFrame(getdata('/Users/rupalgandhre/SpringBoard/DataScience_Capstone2/data/raw/effy_neck_pend.txt'))

df_neck_pend['Jewelry_Type'] = 'Necklace and Pendant'
df_neck_pend['Sex'] = 'Women' 
           


response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-watercolors-14k-white-gold-multi-sapphire-diamond-pendant-1-17-tcw
response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-14k-yellow-gold-diamond-station-necklace-1-35-tcw
response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-signature-14k-white-gold-diamond-panther-necklace-4-51-tcw
response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-brasilica-14k-yellow-gold-emerald-and-diamond-necklace-9-74-tcw
response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-novelty-14k-rose-gold-ruby-and-diamond-evil-eye-pendant-0-39-tcw
response <Response [200]>
url https://www.effyjewelry.com/collections/necklaces-pendants/products/effy-novelty-14k-gold-ruby-sapphire-diamond-eagle-head-pendant-1-06-tcw
response <

In [ ]:
df_neck_pend.head(1)

In [ ]:
df_neck_pend.to_csv("effy_neck_pend.csv", index=False, header=True)


## Concat all the dataframes

In [ ]:


frames = [df_earrings,df_rings,df_bracelets,df_neck_pend]

result = pd.concat(frames)

result.to_csv("effy_all_jewelry_cat.csv", index=False, header=True)